## RNN(Recurrent Neural Network, 순환 신경망)
- 시퀀스 데이터를 모델링하기 위함
- 시퀀스 : 음악, 동영상, 소설, 주가 차트 등 연속적인 시계열 데이터
- 여러 개의 데이터가 순서대로 입력되었을 때 앞서 입력받은 데이터를 잠시 기억해 놓는 방법
- 기억된 데이터가 얼마나 중요한지를 판단하여 별도의 가중치를 줘서 다음 데이터로 넘어감

## LSTM(Long Short Term Memory)
- RNN의 단점을 극복하기 위함
- RNN의 단점 : 시퀀스 중 중요한 입력과 출력 단계의 사이가 멀어질수록 그 관계를 학습하기 어려워짐(기울기 소실 문제)
- 반복되기 직전에 다음 층으로 기억된 값을 넘길지 안넘길지를 관리하는 단계를 하나 더 추가하는 것

https://dreamgonfly.github.io/rnn/2017/09/04/understanding-rnn.html

## LSTM을 이용한 로이터 뉴스 카테고리 분류하기
11,258개의 뉴스 기사를 46개의 카테고리로 분류
https://github.com/gilbutITbook/006958/blob/master/deeplearning/deep_code/17_RNN1_Reuters.py

In [1]:
# 로이터 뉴스 데이터셋 불러오기
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence
from keras.utils import np_utils

import numpy
import matplotlib.pyplot as plt

# seed 값 설정
seed = 0
numpy.random.seed(seed)

In [2]:
# 불러온 데이터를 학습셋, 테스트셋으로 나누기
(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

2113536/2110848 [==============================] - 1s 0us/step


In [7]:
# 데이터 확인하기
category = numpy.max(Y_train) + 1
print(category, '카테고리')
print(len(X_train), '학습용 뉴스 기사')
print(len(X_test), '테스트용 뉴스 기사')
print(X_train[0])

46 카테고리
8982 학습용 뉴스 기사
2246 테스트용 뉴스 기사
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [ ]:
## 기사를 출력해보니 단어가 나오는 게 아니라 숫자가 나옴
## 딥러닝은 단어를 그대로 사용하지 않고 숫자로 변환한 다음 학습할 수 있음
## 여기서는 데이터 안에서 해당 단어가 몇 번이나 나타나는지 세어 빈도에 따라 번호를 붙였음
## 예를 들어, 3이라고 하면 세 번째로 빈도가 높은 단어라는 뜻
## 모든 단어를 다 사용하는 것은 비효율적이므로 빈도가 높은 단어만 불러와 사용할 것임
## 빈도가 1~1000에 해당하는 단어만 선택해서 불러오자
## 주의해야 할 점, 각 기사의 단어 수가 제각각 다르므로 단어의 숫자를 맞춰야 함 -> sequence() 함수 사용

In [9]:
# 데이터 전처리
x_train = sequence.pad_sequences(X_train, maxlen=100)
x_test = sequence.pad_sequences(X_test, maxlen=100)

In [ ]:
## 단어 수를 100개로 맞추는 작업
## 만일 입력된 기사의 단어 수가 100보다 크면 100개째 단어만 선택하고 나머지는 버림
## 100에서 모자를 때는 모자라는 부분을 모두 0으로 채움

In [10]:
# y 데이터 원핫인코딩
y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)

In [12]:
# 모델의 설정
model = Sequential()
model.add(Embedding(1000, 100))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(46, activation='softmax'))

In [ ]:
## Embedding층은 입력된 값을 받아 다음 층이 알아들을 수 있는 형태로 변환하는 역할
## Embedding('불러온 단어의 총 개수', '기사당 단어 수')
## LSTM은 RNN에서 기억 값에 대한 가중치를 제어
## LSTM('기사당 단어 수', '기타 옵션')

In [13]:
# 모델의 컴파일
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [14]:
# 모델의 실행
history = model.fit(x_train, y_train, batch_size=100, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
90/90 [==============================] - 20s 228ms/step - loss: 2.5634 - accuracy: 0.3642 - val_loss: 2.1071 - val_accuracy: 0.4751
Epoch 2/20
90/90 [==============================] - 21s 230ms/step - loss: 1.9227 - accuracy: 0.4973 - val_loss: 1.8173 - val_accuracy: 0.5303
Epoch 3/20
90/90 [==============================] - 20s 221ms/step - loss: 1.8131 - accuracy: 0.5235 - val_loss: 1.7681 - val_accuracy: 0.5396
Epoch 4/20
90/90 [==============================] - 19s 215ms/step - loss: 1.7094 - accuracy: 0.5560 - val_loss: 1.7564 - val_accuracy: 0.5508
Epoch 5/20
90/90 [==============================] - 20s 220ms/step - loss: 1.6670 - accuracy: 0.5780 - val_loss: 1.6753 - val_accuracy: 0.5908
Epoch 6/20
90/90 [==============================] - 20s 225ms/step - loss: 1.5191 - accuracy: 0.6179 - val_loss: 1.5584 - val_accuracy: 0.6118
Epoch 7/20
90/90 [==============================] - 20s 227ms/step - loss: 1.4154 - accuracy: 0.6448 - val_loss: 1.4634 - val_accuracy: 0.6358

In [15]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

71/71 [==============================] - 2s 33ms/step - loss: 1.2295 - accuracy: 0.7137

 Test Accuracy: 0.7137


## LSTM과 CNN의 조합을 이용한 영화 리뷰 분류하기
영화에 대한 2만 5000여 개의 영화 리뷰를 통해 해당 영화를 긍정적으로 평가했는지 부정적으로 평가했는지 감성 분류
https://github.com/gilbutITbook/006958/blob/master/deeplearning/deep_code/17_RNN2_imdb_lstm.py

In [16]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D
from keras.datasets import imdb

import numpy
import matplotlib.pyplot as plt

# seed 값 설정
seed = 0
numpy.random.seed(seed)

In [17]:
# 학습셋, 테스트셋 지정하기
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=5000)

17465344/17464789 [==============================] - 7s 0us/step


In [19]:
y_train # 긍정/부정만 존재 -> 원핫인코딩 과정 필요 없음

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [20]:
# 데이터 전처리
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [21]:
# 모델의 설정
model = Sequential()
model.add(Embedding(5000, 100))
model.add(Dropout(0.5))
model.add(Conv1D(64, 5, padding='valid', activation='relu',strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary() # 모델 구조

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         500000    
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 55)                26400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 56        
_________________________________________________________________
activation (Activation)      (None, 1)                

In [ ]:
## 2차원 배열을 가진 이미지와는 다르게 지금 다루는 데이터는 배열 형태로 이루어진 1차원 데이터임
## Conv1D는 Conv2D의 개념을 1차원으로 옮긴 것
## MaxPooling1D도 마찬가지로 2차원 배열이 1차원으로 바뀌어 정해진 구역 안에서 가장 큰 값을 다음 층으로 넘기고 나머지는 버림

In [22]:
# 모델의 컴파일
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
# 모델의 실행
history = model.fit(x_train, y_train, batch_size=100, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
250/250 [==============================] - 27s 107ms/step - loss: 0.2087 - accuracy: 0.9183 - val_loss: 0.3325 - val_accuracy: 0.8556
Epoch 2/5
250/250 [==============================] - 26s 103ms/step - loss: 0.1743 - accuracy: 0.9345 - val_loss: 0.3591 - val_accuracy: 0.8519
Epoch 3/5
250/250 [==============================] - 26s 104ms/step - loss: 0.1451 - accuracy: 0.9463 - val_loss: 0.3823 - val_accuracy: 0.8518
Epoch 4/5
250/250 [==============================] - 26s 103ms/step - loss: 0.1179 - accuracy: 0.9573 - val_loss: 0.4297 - val_accuracy: 0.8485
Epoch 5/5
250/250 [==============================] - 26s 103ms/step - loss: 0.1033 - accuracy: 0.9637 - val_loss: 0.4641 - val_accuracy: 0.8454


In [25]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 8s 11ms/step - loss: 0.4641 - accuracy: 0.8454

 Test Accuracy: 0.8454
